# A Longest Path Example -- Building a Stadium

We have the following information about the tasks that need to be completed to build a stadium:


|Task | Duration (weeks) | Predecessors |
|----:|---------:|-------------:|
|1|2|none|
|2|16|1|
|3|9|2|
|4|8|2|
|5|10|3|
|6|6|4,5|
|7|2|4|
|8|2|6|
|9|9|4,6|
|10|5|4|
|11|3|6|
|12|2|9|
|13|1|7|
|14|7|2|
|15|4|4,14|
|16|3|8,11,14|
|17|9|12|
|18|1|17|

What is the shortest time in which we could complete the entire stadium? Which tasks are on the so-called "critical path," or the sequence of activities that will take the longest to complete?

Here we will model this finding the maximum length path from start to finish in the project network.
It is again a MCNF model, where nodes are the set of tasks $T$.  There is an arc between tasks $i \in T$ and $j \in T$ if $i$ directly precedes $j$.
$b_i = 1$ for the starting task, and $b_i = -1$ for the finishing task.  (In modeling these in practice, you may need to create a dummy starting task and a dummy finishing tasks, connecting tasks that have no predecessors to the starting task, and connecting tasks that have no antecedents to the finishing task).
The cost of arc $(i,j)$ is the time to complete task $j$.

## Building the Data and Model in Julia 

In [3]:
using JuMP, HiGHS

### DATA ###

# task list
T = [:S, :1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :F]

# each task has a duration
duration = Dict(zip(T,[0,2,16,9,8,10,6,2,2,9,5,3,2,1,7,4,3,9,1,0]))

pred = [(:S,:1), (:1, :2), (:2, :3), (:2,:4), (:2,:14), (:3,:5), (:4,:6), (:4,:7), (:4,:9), (:4,:15), (:5,:6), (:6,:8), (:6,:9), (:6,:11), (:7,:13), (:8,:16), (:9,:12), (:10,:F), (:11,:16), (:12,:17), (:13,:F), (:14,:15), (:14,:16), (:15,:F), (:16,:F), (:17,:18), (:18,:F)] 

# Now we can set the costs 
c = Dict()
for i in T
    for j in T
        if (i,j) in pred
            c[i,j] = duration[j]
        end
    end
end 

# Net supply 
b = Dict(i => 0 for i in T)
b[:S] = 1
b[:F] = -1

# # ### MODEL ###

m = Model(HiGHS.Optimizer)

@variable(m, 0 <= x[pred] <= 1) # arcs in our longest path network

@objective(m, Max, sum(c[a]*x[a] for a in pred))

@constraint(m, flow_balance[k in T], sum(x[(k,j)] for j in T if (k,j) in pred) - sum(x[(i,k)] for i in T if (i,k) in pred) == b[k])


set_silent(m)
# # # solve this instance of the longest path problem
optimize!(m)


println("Longest path (shortest completion time) is ", round(objective_value(m),digits=1))
for (i,j) in pred
    if value(x[(i,j)]) > 0.1
        println(i, ",", j, " is on longest path")
    end 
end 


Longest path (shortest completion time) is 64.0
S,1 is on longest path
1,2 is on longest path
2,3 is on longest path
3,5 is on longest path
5,6 is on longest path
6,9 is on longest path
9,12 is on longest path
12,17 is on longest path
17,18 is on longest path
18,F is on longest path
0; Iter: Time   1.907e-08; average =   1.907e-09; Bound =   1.915e-06
